### Los Bosques Aleatorios y photo-zs (corrimientos al rojo)

En este cuaderno, usaremos Bosques Aleatorios para estimar los 'redshifts' (corrimientos al rojo) fotométricos de galaxias, empezando con observaciones de magnitudes de galaxias en seis diferentes bandas fotométricas. Este cuaderno acompaña Capítulo 6 del libro.

Queremos reproducir o mejorar los resultados de [este artículo](https://arxiv.org/abs/1903.08174) ; sus datos son públicamente disponibles [aquí](http://d-scholarship.pitt.edu/36064/).

Autor: Viviana Acquaviva, con contibuciones de Jake Postiglione y Olga Privman. Traducido por Lucia Perez y Rosario Cecilio-Flores-Elie. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
import astropy

from astropy.io import fits

#fits significa "Flexible Image Transport System" o "Sistema Flexible para Transportar Imágenes"; es un formata que puede guardar imágenes y datos sumarios


### Ingreso de Datos

En mi opinión, lo mas facil es leer los datos con un marco de datos de pandas:

In [ ]:
with fits.open('../data/DEEP2_uniq_Terapix_Subaru_v1.fits') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder()) #mirar https://numpy.org/devdocs/user/basics.byteswapping.html#changing-byte-ordering

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

Puedo escoger las columnas que corresponded al brillo de las galaxias en las seis bandas de interés.

In [ ]:
features = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]

El objetivo es identificar la caracteristica del corrimiento al rojo. Para este catologo, los corrimientos al rojo espectroscópicos (que son más precisos) están disponibles en esta columna:

In [ ]:
target = df['zhelio']

In [ ]:
features.head(10)

In [ ]:
target.head(10)

### ¡Ahora podemos empezar nuestro primer modelo de Bosque Aleatorio!

Para entender lo que queremos, miremos esta figura del artículo:

 ![Desempeño de la reconstruction de los corrimientos al rojo fotométricos](Photoz_RF_CFHTLS_Deep.png)

En esta figura, $\sigma_{NMAD}$ es la mediana normalizada de la desviación absoluta del vector residual; y $\eta$ es la fracción de valores atípicos, definidos como esos por cual (z_verdad - z_estimado)/(1+z_verdad) > 0.15.

Para ser justos, estamos usando datos de DEEP2/3, entonces nuestro rango es un poco diferente.

In [ ]:
model = RandomForestRegressor()

In [ ]:
model.get_params()

Establecer el punto de referencia.

In [ ]:
scores = cross_validate(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
scores

¡Mosca que toma tiempo! Y las notas son las de R2 en este momento.

In [ ]:
np.mean(scores['test_score'])

In [ ]:
np.mean(scores['train_score'])

### Revisión de aprendizaje
    
What issue do these scores indicate?

¿Qué problema está indicado en estas notas?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```
Parece que tenemos un gran problema de alta divergencia.
```
    
</p>
</details>

¿Por qué?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>

```

Se ve una gran diferencia entre las notas de entrenamiento y de prueba. Para ser justos, debemos también mirar a la desviación estándar de las notas de entrenamiento y de prueba para confirmar que la diferencia es significativa.

```
    
</p>
</details>
</br>


Vamos también a ver a las predicciones:

In [ ]:
ypred = cross_val_predict(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.scatter(target,ypred, s = 20, c = 'royalblue')
plt.xlabel('True (spectroscopic) z', fontsize=14)
plt.ylabel('Predicted z',fontsize=14)
plt.axis('square')
plt.xlim(0,3)
plt.ylim(0,3)

### Pregunta: ¿Se parecen a las del artículo?

También es interesante ver la distribución de los valores previstos, y ver que casi siempre producen una distribución más angosta. ¿Por qué?

In [ ]:
plt.hist(target,bins=50,density=False,alpha=0.5, range = (0,3), label = 'True');
plt.hist(ypred,bins=50,density=False,alpha=0.5, range = (0,3), color = 'g', label = 'Predicted');
plt.legend(fontsize=14);

Ahora podemos calcular la fracción de valores atípicos:

In [ ]:
len(np.where(np.abs(target-ypred)>0.15*(1+target))[0])/len(target)

Y el NMAD ("normalized median absolute deviation"):

In [ ]:
1.48*np.median(np.abs(target-ypred)/(1 + target)) 
# Para una distribución Gaussiana, esto se hace una desviación estándar--por eso el 1.48

### Tenemos un gran problema de muy alta divergencia, entonces podemos optimizar nuestros parámetros. 

Primero podemos reducir el tamaño del conjunto de datos, especialmente porque ya vimos lo lento que fue usar los k-fold CV simples.

In [ ]:
np.random.seed(20)
sel = np.random.choice(range(len(ypred)), 5000, replace = False) #tomar muestras sin reemplazarlas

In [ ]:
len(np.unique(sel))

Creamos un conjunto de datos más pequeño:

In [ ]:
seld = features.loc[sel,:]
selt = target[sel]

In [ ]:
littlescores = cross_validate(model,seld,selt, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
littlescores['test_score'].mean(), littlescores['train_score'].mean()

Que el desempeño es similar con el conjunto de entrenamiento nos asegura que el tamaño del conjunto no es un gran problema, y podemos seguir optimizando.

#### parámetros del árbol

Los parámetros asociados con esto son:

- El número mínimo de casos en nódulo de hoja;

- El número mínimo de casos requeridos en un nódulo partido; 

- La profundidad máxima del árbol;

- El criterio que decide si un partido "vale la pena", que se expresa en términos del gano de información.


#### parámetros del Aleatorización

- El número de carácteristicas k < n que se usan en construir árboles;

- El valor de retomar muestras ("bootstrap") del conjunto de datos (T or F, Verdad o Falso).

#### parámetros del Bosque

El número de árboles en el bosque (n_estimadores) se puede ajustar; se entiende que más árboles es mejor, pero eventualmente el desempeño se estabilizará, entonces podemos balancear el número de árboles y la duración del cada ronda (de entrenamiento, optimización, etc.).

In [ ]:
model.get_params()

Imaginemos un conjunto posible.

- min_impurity_decrease (reducción mínima de la impureza)

- número de Árboles
 
- max_leaf_nodes (número maxímo de nódulos de hoja)

- min_samples_split (número mínimo de conjuntos después de partirlos)

- max_features (número maximo de carácteristicas)

In [ ]:
# Dura unos minutos terminar

parameters = {'min_impurity_decrease':[0.1, 0.5, 0.0], \
              'max_features':[None,4,2], 'n_estimators':[50, 100, 200], 'min_samples_split': [10,20,100], 
              'max_leaf_nodes':[None, 100, 200]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(seld,selt)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

print('Los mejores parámetros, la mejor nota:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)


In [ ]:
scoresCV

### Y el resultado es...

NO mejoramos las notas de prueba.

### ¡Es el momento de considerar cómo limpiar o entrar los datos!

En mi caso, tuve que escribirles a los autores del artículo original. Me dijeron exactamente cómo escogieron los datos para el conjunto de entrenamiento. 

In [ ]:
mags = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor','subaru_source','cfhtls_source','zquality']]

In [ ]:
mags.head()

In [ ]:
mags.shape

In [ ]:
# calidad del corrimiento al rojo - queremos solo usar objetos que tienen corrimientos al rojo (espectroscópicos) de alta calidad

mags = mags[mags['zquality'] >= 3]

mags.shape

In [ ]:
#foto de cfhtls (profunda)

mags = mags[mags['cfhtls_source'] == 0]

mags.shape

In [ ]:
# No lo usaremos por ahora, pero esta es la foto profunda de subaru

#mags = mags[mags['subaru_source'] == 0]

#mags.shape

Medidas no disponibles se señalan con -99 o 99 (valores típicos son 20-25). Podemos botar datos sin medidas.

In [ ]:
mags = mags[mags > -10].dropna()

In [ ]:
mags.shape

In [ ]:
mags = mags[mags < 90].dropna()

In [ ]:
mags.shape

Nuestro conjunto final tiene 6,307 objetos y usa las 6 carácteristicas originales.

In [ ]:
sel_features = mags[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]
sel_features.head()

Escogemos la misma colección en el vector de meta.

In [ ]:
sel_target = target[sel_features.index]

Vamos a ver cómo le va a nuestro modelo de referencia. Para tener resultados duplicables, tenemos que fijar el parámetro "random\_state" (estado aleatorio, que controla el procesador de bootstrap) del Bosque Aleatorio, y la semilla aleatoria de la validación cruzada. 

In [ ]:
scores = cross_validate(RandomForestRegressor(random_state = 5),sel_features,sel_target,cv = KFold(n_splits=5, shuffle=True, random_state=10), \
               return_train_score=True)

In [ ]:
print(np.round(np.mean(scores['test_score']),3), np.round(np.std(scores['test_score']),3))

In [ ]:
print(np.round(np.mean(scores['train_score']),3), np.round(np.std(scores['train_score']),3))

¡Las notas si se mejoran! Pero, todavía se ve alta divergencia. Podemos rehacer el proceso de optimización (mosca que el tamaño del conjunto de datos ya está limitado, entonces no se tiene que encoger).

In [ ]:
#Esto me tomó ~3 minutos

parameters = {'max_depth':[3, 6, None], \
              'max_features':[None,4,2], 'n_estimators':[50,100,200], 'min_samples_leaf': [1,5,10]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(random_state = 5), parameters, cv = KFold(n_splits=5, shuffle=True, random_state=10), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV

### Revisión de aprendizaje

Viendo los resultados de la búsqueda en cuadrícula, ¿esperarías que el desempeño se mejoraría mucho si se agranda el espacio de los parámetros?

<br>
<details><summary><b>¡Haga clic aquí para obtener la respuesta!</b></summary>
<p>
    
```
Probablemente no, porque las notas no cambian mucho en los primero 10-20 modelos; eso sugiere que seguir optimizando no las mejoraría mucho.
```
    
</p>
</details>

In [ ]:
bm = model.best_estimator_

Se puede generar una tanda de predicciones para visualizar lo que pasaría.

In [ ]:
ypred = cross_val_predict(bm, sel_features,sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.xlabel('z_spec')
plt.ylabel('z_photo')
plt.ylim(0,2)
plt.xlim(0,2)

Calculemos la fracción de valores atípicos y comparemos con la figura.

In [ ]:
len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)

Calculemos desviación absoluta mediana normalizada (Normalized Median Absolute Deviation, NMAD)

In [ ]:
1.48*np.median(np.abs(sel_target-ypred)/(1 + sel_target))

### Conclusión: ¿cómo se compara nuestro modelo con el del artículo?

El nuestro es un poco peor, pero tenemos una ventaja secreta en la ingeniería de características.

### Ejercicio en la ingeniería de Características: ¿qué pasaría si usamos los colores en lugar de las magnitudes?

In [ ]:
sel_features.loc[:,'u-g'] = sel_features['u_apercor'] - sel_features['g_apercor']
sel_features.loc[:,'g-r'] = sel_features['g_apercor'] - sel_features['r_apercor']
sel_features.loc[:,'r-i'] = sel_features['r_apercor'] - sel_features['i_apercor']
sel_features.loc[:,'i-z'] = sel_features['i_apercor'] - sel_features['z_apercor']
sel_features.loc[:,'z-y'] = sel_features['z_apercor'] - sel_features['y_apercor']

In [ ]:
sel_colors = sel_features[['u-g','g-r','r-i','i-z','z-y','i_apercor']]

In [ ]:
scores = cross_validate(RandomForestRegressor(),sel_colors,sel_target,cv = KFold(n_splits=5, shuffle=True, random_state=10), \
               return_train_score=True)

In [ ]:
scores 

In [ ]:
scores['test_score'].mean(), scores['test_score'].std()

In [ ]:
parameters = {'max_depth':[3, 6, None], \
              'max_features':[None,4,2], 'n_estimators':[50,100,200], 'min_samples_leaf': [1,5,10]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_colors, sel_target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV

In [ ]:
bm

In [ ]:
bm = model.best_estimator_

In [ ]:
ypred = cross_val_predict(bm, sel_colors, sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

### Revisión de aprendizaje
    
Calcula el NMAD y la fracción de valores atípicos en los corrimientos al rojo predichos comparados con los verdaderos, completando este programa.

```python
1.48 * np.median(... (... - ...)/(1 + ...))

len(... (np.abs(...) > ... * (1 + ...))[0]) / len(...)
```

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
    
```python
1.48 * np.median(np.abs(sel_target-ypred)/(1 + sel_target))

len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)
```
    
</p>
</details>
</br>

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.ylim(0,2)
plt.xlim(0,2)

¡Por fin logramos el desempeño del artículo! (Pero mosca que no estamos usando *exactamente* los mismos datos.)

### <font color='blue'> Antes de hoy, no hemos hablado de algo muy importante: cómo estimar la incertidumbre asociada con nuestros resultados.</font>

Una fuente de la dispersión viene de las métricas de desempeño global es la arquitectura de nuestra sistema: deberíamos generar muchas predicciones con muchas diferentes semillas aleatorias. Esto es el equivalente a la dispersión que se ve en las métricas monitorizadas (p.ej. notas de MSE o r2) en la validación cruzada.

In [ ]:
model = RandomForestRegressor(max_features=4, n_estimators=200) # tengo que "re-sembrar" el estado aleatorio

In [ ]:
# Mosca, esto también es lentoooooo

seeds = np.random.choice(100,8, replace = False) #pick 8

olf = np.zeros(8)
NMAD = np.zeros(8)

for i in range(8): # Un poco básico, pero nos medio-muestra que pasa cuando se cambian las semillas aleatorias
    print('Iteration', i)
    ypred = cross_val_predict(RandomForestRegressor(max_features=4, n_estimators=200,random_state=seeds[i]), sel_features, sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=seeds[i]))
    olf[i] = len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)
    NMAD[i] = 1.48*np.median(np.abs(sel_target-ypred)/(1 + sel_target))

print('OLF avg/std:, {0:.5f}, {1:0.5f}'.format(olf.mean(), olf.std()))
print('NMAD avg/std:, {0:.5f}, {1:0.5f}'.format(NMAD.mean(), NMAD.std()))

#### Sin embargo, tenemos que pensar cómo cuantificar el error observacional en cada una de nuestras entradas *individualmente*. 

No existe mucha literatura que explora esto, pero propongo un "forward-pass" (pase adelantado): ejecutamos el mejor modelo muchas veces, cada vez con entradas diferentes que obtenemos al modelar sus perfiles de ruido (p.ej. un Gaussian por cual el mediano es el valor medido, y sigma es el error experimental).

Si supongamos quel perfil de ruido no cambia del valor "verdadero" al valor "observado", el debe incluir el error experimental y el error de "información limitada", que viene de tener un conjunto de entrenamiento limitado, carácteristicas no informativas, la arquitectura del modelo, etc. Esto se describe como incertidumbre "epistemológica" o "aleatorica" (p.ej. [esta reseña reciente](https://link.springer.com/article/10.1007/s10994-021-05946-3)).